In [1]:
import pandas as pd

df_covid = pd.read_csv("KorBertSum/test_content_b_1.csv").drop('Unnamed: 0', axis=1)
df_covid.head()

,title,content,date,time,url,index
0,"LPGA, 스웨덴 코크 생애 두번째 우승",스웨덴의 미녀 골프 선수 카린 코크가 생애 두번째우승컵을 품에 안았다.\n\n코크는...,2005.4.25,오전 7시 28분 0초,https://www.hani.co.kr/arti/sports/golf/29025....,27248
1,LPGA 별들 한국그린서 대격돌,"19일 개막 엑스켄버스오픈 출전\n\n제니퍼 로살레스(필리핀), 로레나 오초아(멕시...",2005.5.18,오후 6시 16분 0초,https://www.hani.co.kr/arti/sports/golf/34933....,27247
2,"고압선 근처 사는 아이들, 백혈병 위험",고압선 부근에 사는 아이들은 백혈병 위험이 높다는 연구결과가 나왔다.\n\n영국 옥...,2005.6.3,오후 4시 7분 0초,https://www.hani.co.kr/arti/society/health/393...,27246
3,"위성미, 티타임에서도 ‘유명 프로’ 대우",10월 프로 전향설이 유력하게 나돌고 있는 가운데 위성미(15.미셸 위)가 처음 출...,2005.7.27,오전 10시 25분 56초,https://www.hani.co.kr/arti/sports/golf/53030....,27242
4,백두산 상공에 UFO 출현?,백두산 상공에 미확인비행물체(UFO)가 출현했다는 주장이 제기됐다.\n\n2일 동포...,2005.8.2,오후 1시 58분 30초,https://www.hani.co.kr/arti/society/society_ge...,27241


In [3]:
idxs = df_covid['index'].tolist()
texts = df_covid['content'].tolist()
titles = df_covid['title'].tolist()
dates = df_covid['date'].tolist()
urls = df_covid['url'].tolist()
idx_text = list(zip(idxs, texts, titles, dates, urls))

In [4]:
import requests
import json
from tqdm import tqdm

In [5]:
for i in tqdm(range(len(idx_text))):
    idx_text[i] = list(idx_text[i])

100%|██████████| 27134/27134 [00:00<00:00, 1068421.37it/s]


In [6]:
# for i in tqdm(range(len(text))):
#     resp = requests.get("http://127.0.0.1:5000/predict?text=" + text[i])
# #     print(resp)
#     print(json.loads(resp.content))

In [7]:
# !git clone https://github.com/hsh2438/mnist_serving_tf2.0_flask.git

In [8]:
# !pip install -r mnist_serving_tf2.0_flask/requirements.txt

In [9]:
# !pip install gunicorn

In [10]:
# !pip install fake_useragent

In [6]:
# ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

def inference(texts):
    
    global df_covid
    idx = texts[0]
    text = texts[1]
    title = texts[2]
    date = texts[3]
    url = texts[4]
    df = df_covid[df_covid['index'] == idx]
    index = df.index[0]
    
    if (text != ""):
        headers = {'Content-Type':'application/json', 
                   'User-Agent' : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"
                   ,'Connection':'close'
                  }
        address = "http://127.0.0.1:5000/predict"
        data = {'text' : text }
        
        try:
            resp = requests.get(address, data=json.dumps(data), headers=headers)
            
            new_list.append(
                {
                    "index" : idx,
                    "title" : title,
                    "content" : text,
                    "date" : date,
                    "url" : url,
                    "summary" : str(resp.content, encoding='utf-8')
                }
            )
            resp.close()
        except requests.exceptions.Timeout as errd:
            print("Timeout Error : ", errd)

        except requests.exceptions.ConnectionError as errc:
            print("Error Connecting : ", errc)

        except requests.exceptions.HTTPError as errb:
            print("Http Error : ", errb)

        # Any Error except upper exception
        except requests.exceptions.RequestException as erra:
            print("AnyException : ", erra)
            
#         print(str(resp.content, encoding='utf-8'))
    else:
        new_list.append(
            {
                "index" : idx,
                "title" : title,
                "content" : text,
                "date" : date,
                "url" : url,
                "summary" : ""
            }
        )
    df_covid = df_covid.drop(index, axis=0)
    return ''

In [7]:
from multiprocessing.dummy import Pool as ThreadPool

pool = ThreadPool(10)

In [ ]:
new_list = []

# for _ in tqdm(pool.imap_unordered(inference, idx_text), total=len(idx_text)):
#     pass
    
# print("finish")
# ConnectionError

try:
    for _ in tqdm(pool.imap_unordered(inference, idx_text), total=len(idx_text)):
        pass
    
    print("finish")
    
except ConnectionError:
    pool.close()
    pool.terminate()
    pool.join()
    
#     df_covid.to_csv("../코로나_content_b_9.csv")
    
#     col_name = ["index", "title", "content", "date", "url", "summary"]
#     news_df = pd.DataFrame(new_list, columns=col_name)

#     csv_file_name = "../한겨레_covid_summary_b_9.csv" # 이름 바꿔줘야함

#     news_df.to_csv(csv_file_name)
#     news_df.head()

In [ ]:
# pool.close()
# pool.terminate()
# pool.join()
    
df_covid.to_csv("test.csv")

col_name = ["index", "title", "content", "date", "url", "summary"]
news_df = pd.DataFrame(new_list, columns=col_name)

csv_file_name = "test_2.csv" # 이름 바꿔줘야함

news_df.to_csv(csv_file_name)
news_df.head()